In [1]:
import pandas as pd
import nltk
import re 
from nltk.tokenize import sent_tokenize, word_tokenize

# Challenge 2

In [2]:
def clean_up(s):
    
        s=re.sub(r"http\S+", "", s)
        s=re.sub(r"\d{1,}", "", s)
        s=re.sub(r'[^a-zA-Z]', " ", s)
    
        return s.lower()

    
clean_up("@Ironhack's-#Q website 776-is http://ironhack.com [(2018)]')")

' ironhack s  q website  is        '

In [3]:
def tokenize(s):
    
    s=word_tokenize(s)
    return s

tokenize("ironhack s  q website  is")

['ironhack', 's', 'q', 'website', 'is']

In [4]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

def stem_and_lemmatize(l):
    ps = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stem=[]
    lem=[]
    for i in l:
        stem.append(ps.stem(i))
    for i in stem:
        lem.append(lemmatizer.lemmatize(i))

    return lem

l=['ironhack', 's', 'q', 'website', 'is'] 
l=stem_and_lemmatize(l)
l

['ironhack', 's', 'q', 'websit', 'is']

In [5]:
from nltk.corpus import stopwords
def remove_stopwords(l):
    stopWords = set(stopwords.words('english'))
    wordsFiltered = []   
    for word in l:
        if word not in stopWords:
            wordsFiltered.append(word)

    return wordsFiltered


remove_stopwords(l)

['ironhack', 'q', 'websit']

# Challenge 3

In [6]:
import zipfile
zf = zipfile.ZipFile('Sentiment140.csv.zip') 
sentiment = pd.read_csv(zf.open('Sentiment140.csv'))
# , index_col='CensusId'

In [7]:
#sentiment=sentiment[0:50000]
sentiment = sentiment.sample(n=2500)

In [8]:
sentiment['text processed']=sentiment['text'].apply(clean_up)
sentiment['text processed']=sentiment['text processed'].apply(tokenize)
sentiment['text processed']=sentiment['text processed'].apply(stem_and_lemmatize)
sentiment['text processed']=sentiment['text processed'].apply(remove_stopwords)
sentiment.head()

,target,id,date,flag,user,text,text processed
381414,0,2052780313,Sat Jun 06 01:59:55 PDT 2009,NO_QUERY,sophieholly,@sofiesunshine me too bless her!,"[sofiesunshin, bless]"
887490,4,1687011322,Sun May 03 07:30:03 PDT 2009,NO_QUERY,Kiaroux,Is craving some sushi .. and will tell it to a...,"[crave, sushi, tell, anyon, listen, one, give,..."
1209630,4,1988957701,Sun May 31 23:53:33 PDT 2009,NO_QUERY,mirandaforwood,"(and yes, I know I'm slow to the party here","[ye, know, slow, parti]"
198271,0,1971182522,Sat May 30 06:26:48 PDT 2009,NO_QUERY,ZombieAlice,Working till 6,"[work, till]"
156528,0,1956136525,Thu May 28 21:15:30 PDT 2009,NO_QUERY,EirelavMajere,wishing 4 things I can't have,"[wish, thing]"


In [9]:
# Combining all words 
all_words=[]
for i in sentiment['text processed']:
    for word in i:
        all_words.append(word)
        


In [10]:
from nltk import FreqDist
freqDist = FreqDist(all_words).most_common(3000)
top_3000 = [word[0] for word in freqDist]



In [49]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def find_features(document):
    words = set(document)
    features = {}
    for w in top_3000:
        features[w] = (w in words)
    analyzer = SentimentIntensityAnalyzer()
    sent=analyzer.polarity_scores(" ".join(document))
    if sent["pos"] > 0.5:
        sent = 'Pos'
    else:
        sent = 'Neg'
    return features, sent




featuresets= list(sentiment['text processed'].apply(find_features))


In [50]:
len(featuresets[2000:])

500

In [52]:
train_set, test_set = featuresets[:2000], featuresets[2000:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
 	
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(15)

0.862
Most Informative Features
                    luck = True              Pos : Neg    =     16.7 : 1.0
            followfriday = True              Pos : Neg    =     14.0 : 1.0
                      ff = True              Pos : Neg    =     13.0 : 1.0
                   thanx = True              Pos : Neg    =     13.0 : 1.0
                 perfect = True              Pos : Neg    =     13.0 : 1.0
                     btw = True              Pos : Neg    =     13.0 : 1.0
                    lone = True              Pos : Neg    =     13.0 : 1.0
                    safe = True              Pos : Neg    =     13.0 : 1.0
                   laugh = True              Pos : Neg    =     13.0 : 1.0
              davidarchi = True              Pos : Neg    =     13.0 : 1.0
                   black = True              Pos : Neg    =     13.0 : 1.0
                   award = True              Pos : Neg    =     10.9 : 1.0
                   cheer = True              Pos : Neg    =     10.0